# Practica Final UD2 SAA

## Datos base

In [2]:
import pandas as pd
import numpy as np

datos_ventas = {
    'Venta_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10], # Hay una fila duplicada
    'Fecha': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02', '2023-01-03', '2023-01-08', '2023-01-08', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-11'],
    'Producto_ID': [101, 102, 101, 103, 104, 101, 102, 103, 102, 101, 101],
    'Vendedor_ID': [1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1],
    'Cantidad': [20, 5, 25, 7, 10, 18, 6, 8, np.nan, 15, 15], # Hay un valor nulo
    'Precio_str': ['$ 5.00', '€ 20.50', '$ 5.50', '$ 8.00', '€ 3.00', '$ 5.25', '$ 21.00', '$ 8.50', '€ 22.00', '$ 5.00', '$ 5.00'],
    'Sucursal': ['Norte', '  Sur', 'norte', 'Norte', 'Sur', 'Norte', 'Sur', 'Norte', 'Sur', '  norte  ', '  norte  '] # Nombres inconsistentes
}

datos_productos = {
    'Producto_ID': [101, 102, 103, 104],
    'Nombre_Producto': ['Café', 'Tarta', 'Pastel', 'Té'],
    'Categoría': ['Bebida', 'Postre', 'Postre', 'Bebida']
}

datos_vendedores = {
    'Vendedor_ID': [1, 2],
    'Nombre_Vendedor': ['Ana', 'Luis']
}

## Limpieza de datos

In [6]:
df_ventas = pd.DataFrame(datos_ventas)
df_productos = pd.DataFrame(datos_productos)
df_vendedores = pd.DataFrame(datos_vendedores)

df_ventas = df_ventas.drop_duplicates()

df_ventas = df_ventas.dropna()

df_ventas['Sucursal'] = df_ventas['Sucursal'].str.strip().str.lower()

df_ventas['Precio_str'] = df_ventas['Precio_str'].str.replace('[^0-9.]', '', regex=True).astype(float)

df_final = df_ventas.merge(df_productos, on='Producto_ID', how='left')
df_final = df_final.merge(df_vendedores, on='Vendedor_ID', how='left')

df_final['Ingresos_Totales'] = df_final['Cantidad'] * df_final['Precio_str']

df_final['Fecha'] = pd.to_datetime(df_final['Fecha'])
df_final = df_final.set_index('Fecha')

print(df_final.head())

            Venta_ID  Producto_ID  Vendedor_ID  Cantidad  Precio_str Sucursal  \
Fecha                                                                           
2023-01-01         1          101            1      20.0         5.0    norte   
2023-01-01         2          102            2       5.0        20.5      sur   
2023-01-02         3          101            1      25.0         5.5    norte   
2023-01-02         4          103            1       7.0         8.0    norte   
2023-01-03         5          104            2      10.0         3.0      sur   

           Nombre_Producto Categoría Nombre_Vendedor  Ingresos_Totales  
Fecha                                                                   
2023-01-01            Café    Bebida             Ana             100.0  
2023-01-01           Tarta    Postre            Luis             102.5  
2023-01-02            Café    Bebida             Ana             137.5  
2023-01-02          Pastel    Postre             Ana              5

## Análisis Exploratorio y Creación de Variables

In [14]:
ingresos_categoria_vendedor = df_final.groupby(['Categoría', 'Nombre_Vendedor'])['Ingresos_Totales'].sum()
print("Ingresos totales por Categoría y por Vendedor:")
print(ingresos_categoria_vendedor, '\n')

cantidad_categoria_sucursal = df_final.pivot_table(values='Cantidad', index='Categoría', columns='Sucursal', aggfunc='sum')
print("Cantidad total de productos vendidos por Categoría y Sucursal:")
print(cantidad_categoria_sucursal, '\n')

df_final['Tipo_Venta'] = np.where(df_final['Ingresos_Totales'] > 50, 'Grande', 'Normal')

producto_estrella = (
    df_final.groupby(['Categoría', 'Nombre_Producto'])['Ingresos_Totales']
    .sum()
    .reset_index()
)
producto_estrella = producto_estrella.loc[
    producto_estrella.groupby('Categoría')['Ingresos_Totales'].idxmax()
]
print("Producto Estrella por Categoría:")
print(producto_estrella.reset_index(drop=True), '\n')

df_final['Pct_Ingreso_Categoria'] = (
    df_final['Ingresos_Totales'] / df_final.groupby('Categoría')['Ingresos_Totales'].transform('sum') * 100
)

print("DataFrame final con nuevas columnas:")
print(df_final[['Nombre_Producto', 'Categoría', 'Nombre_Vendedor', 'Ingresos_Totales', 'Tipo_Venta', 'Pct_Ingreso_Categoria']].head())

Ingresos totales por Categoría y por Vendedor:
Categoría  Nombre_Vendedor
Bebida     Ana                407.0
           Luis                30.0
Postre     Ana                124.0
           Luis               228.5
Name: Ingresos_Totales, dtype: float64 

Cantidad total de productos vendidos por Categoría y Sucursal:
Sucursal   norte   sur
Categoría             
Bebida      78.0  10.0
Postre      15.0  11.0 

Producto Estrella por Categoría:
  Categoría Nombre_Producto  Ingresos_Totales
0    Bebida            Café             407.0
1    Postre           Tarta             228.5 

DataFrame final con nuevas columnas:
           Nombre_Producto Categoría Nombre_Vendedor  Ingresos_Totales  \
Fecha                                                                    
2023-01-01            Café    Bebida             Ana             100.0   
2023-01-01           Tarta    Postre            Luis             102.5   
2023-01-02            Café    Bebida             Ana             137.5   
2023